In [38]:
import numpy as np

In [39]:
# creates dictionary (used to calculate prob using naive bayes)

def fit(X_train,Y_train):
    
    result = {}
    result['total_data'] = len(Y_train)
    class_values = set(Y_train)
    
    for current_class in class_values:
        
        result[current_class] = {}
        
        current_class_rows = (Y_train==current_class)
        
        X_train_current = X_train[current_class_rows]
        Y_train_current = Y_train[current_class_rows]
        
        
        num_features = X_train.shape[1]
        
        result[current_class]["total_count"] = len(Y_train_current)
        
        for j in range(1,num_features+1):
            result[current_class][j]={}
            
            all_possible_values = set(X_train[:,j-1])
            
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (X_train_current[:,j-1]==current_value).sum()
                
    
    return result;
    

In [40]:
# use log probability to avoid probabiltiy becoming 0 , for large input (as each individual probability will be very small , Product of these prob will become zero)

def probability(dictionary,x,current_class):
    
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary['total_data'])
    num_features = len(dictionary[current_class].keys())-1  #subtract 1 as we have extra key as total_count
    
    for j in range(1,num_features+1):
        
        xj = x[j-1]   # get value of jth feature
        
        count_current_class_with_value_xj = dictionary[current_class][j][xj]+1  # +1 for laplace correction
        count_current_class = dictionary[current_class]["total_count"]+len(dictionary[current_class][j].keys())  # len - count of labels in jth feature for laplace correction
        
        current_xj_probability = np.log(count_current_class_with_value_xj)- np.log(count_current_class)
        
        output = output + current_xj_probability   # changed to log prob
        
    return output


In [41]:
def predictSinglePoint(dictionary,x):
    
    classes = dictionary.keys()
    best_p=-1000
    best_class = -1
    
    first_run = True
    
    
    for current_class in classes:
        
        if current_class == "total_data":
            continue
        
        p_current_class = probability(dictionary,x,current_class)
        if(first_run or p_current_class>best_p):
            best_p = p_current_class
            best_class = current_class
            
        first_run = False
    
    return best_class
    

In [42]:
def predict(dictionary,X_test):
    y_pred = []
    
    for x in X_test:
        x_class = predictSinglePoint(dictionary,x)
        y_pred.append(x_class)
    
    return y_pred

# changing continuous dataset to discrete dataset

In [43]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5*second_limit
    third_limit = 1.5*second_limit
    
    for i in range(0,len(column)):
        
        if(column[i]<first_limit):
            column[i] = 0
        elif(column[i]<second_limit):
            column[i] = 1
        elif(column[i]<third_limit):
            column[i] = 2
        else:
            column[i] = 3
    
    return column
            

In [107]:
from sklearn import datasets
iris = datasets.load_iris()
X= iris.data
Y= iris.target

In [108]:
for i in range(0,X.shape[1]):
    X[:,i] = makeLabelled(X[:,i])

In [109]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)


In [89]:
dictionary = fit(X_train,Y_train)

In [90]:
Y_pred =predict(dictionary,X_test)

In [91]:
from sklearn.metrics import classification_report , confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]


# Using Sklearn Naive Bayes 

In [123]:
from sklearn import datasets
iris = datasets.load_iris()
X= iris.data
Y= iris.target

In [124]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)


In [125]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train,Y_train)

GaussianNB()

In [126]:
Y_pred_2 = clf.predict(X_test)

print(classification_report(Y_test,Y_pred_2))
print(confusion_matrix(Y_test,Y_pred_2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00         9

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38

[[13  0  0]
 [ 0 16  0]
 [ 0  0  9]]


In [ ]:
# in iris GuassianNb  is not performing well compared to our Navie Bayes.
# Because iris dataset does not follow Gaussian curve   